In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
from aerb_unv_reader import UNV

In [3]:
from material_prop_reader import material_section_read

In [4]:
EPR_geo = UNV('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/CRUAS_NPP.unv')
EPR_mat = material_section_read()

Execution Time: 0.5815982818603516 seconds for 2411
Execution Time: 0.5992558002471924 seconds for 2412
Execution Time: 159.03763937950134 seconds for 2477
You are rewriting BAN_e30
You are rewriting BAN_D20Q2
You are rewriting EV
You are rewriting EP
You are rewriting Nu
You are rewriting E_EC_RAD
You are rewriting E_EC_DAL
You are rewriting E_EC_VOIL
TYPE_OF_MODELIZATION;EPAIS;GROUP MESH;EXCENTREMENT
TYPE_OF_MODELIZATION;EPAISSEUR;GROUP MESH
TYPE_OF_MODELIZATION;GROUP MESH;EPAISSEUR;VECTEUR
TYPE_OF_MODELIZATION;GROUP MESH;EPAISSEUR;VECTEUR
TYPE_OF_MODELIZATION;GROUP MESH;EPAISSEUR;VECTEUR
GROUP OF KNOTS_1;GROUP OF KNOTS_2;DDL_1;DDL_2;COEF_MULT_1;COEF_MULT_2
BD4_N2;BD4_N17;DX;DX;1.;-1.
BD4_N2;BD4_N17;DY;DY;1.;-1.
TYPE ON CONNEXION; GROUP OF KNOTS
LIAISON_ELEM;BD5_BASE_BROCHETTE;BD5_CL;PLAQ_POUT_ORTH
BD6_BASE_BROCHETTE;BD6_CL;PLAQ_POUT_ORTH
BD3_BASE_BROCHETTE;BD3_CL;PLAQ_POUT_ORTH
BD7_BASE_BROCHETTE;BD7_CL;PLAQ_POUT_ORTH
BD2_BASE_BROCHETTE;BD2_CL;PLAQ_POUT_ORTH
BD1_BASE_BROCHETTE;BD1_C

In [5]:
FreeNodes = np.loadtxt('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/FreeNodes.dat', dtype=int, delimiter=',')

In [6]:
FreeNodes

array([  2,   3,   4,   5,   6,   7,   8,   9,  27,  28,  29,  30,  31,
        32,  33,  34,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  72,  73, 106, 107, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 204, 205,
       208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
       221, 222, 227, 228, 261, 262, 265, 266, 267, 268, 269, 270, 271,
       272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284,
       285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 29

# Converting Geometries

## Nodes

In [7]:
abaqus_node_str = '** This is Auto generated nodal details\n*Node\n'
for key, XYZ in EPR_geo.fem['nodes'].items():
    if key in FreeNodes:
        continue
    abaqus_node_str = abaqus_node_str + '{0:d},{1:f},{2:f},{3:f}\n'.format(key,XYZ[0],XYZ[1],XYZ[2])

## Elements

In [8]:
## READ ALL ELEMENTS
ELEMENTS = {}
for key,value in EPR_geo.fem['elements'][161].items():
    ELEMENTS[key] = {'type':161, 'connectivity':value}
for key,value in EPR_geo.fem['elements'][21].items():
    ELEMENTS[key] = {'type':21, 'connectivity':value[-2:]}
for key,value in EPR_geo.fem['elements'][91].items():
    ELEMENTS[key] = {'type':91, 'connectivity':value}
for key,value in EPR_geo.fem['elements'][94].items():
    ELEMENTS[key] = {'type':94, 'connectivity':value}

### Extracting List of Beams, ROD and Springs elements from UNV 21 Type Element

In [9]:
GROUPS = {}
for gid, group in EPR_geo.fem['groups'].items():
    base = group['group_data'][0][0]
    for entry in group['group_data']:
        if not(base == entry[0]):
            print(group['group_data'])
    if base == 7:
        typp = 'nodes'
    elif base == 8:
        typp = 'elements'
    GROUPS[group['group_name']] = {'type': typp, 'group': group['group_data'][:,1]}

In [10]:
### Writing ROD / Truss Elemnts File
abaqus_truss_str = '** This is Auto generated truss element details\n*Element, type=T3D2\n'
ROD_ELEMENTS = []
for key,value in EPR_mat.section_details['ROD'].items():
    ROD_ELEMENTS = ROD_ELEMENTS + list(GROUPS[key]['group'])
    # abaqus_mass_str = abaqus_mass_str + '{0:d},{1:d}\n'.format(key,value[0])
ROD_ELEMENTS = sorted(set(ROD_ELEMENTS))
for elem in ROD_ELEMENTS:
    if not(ELEMENTS[elem]['type'] == 21):
        print('Wrong element type {0:d}'.format(elem))
        continue
    abaqus_truss_str = abaqus_truss_str + '{0:d},{1:d},{2:d}\n'.format(elem,\
                                                ELEMENTS[elem]['connectivity'][0],\
                                                ELEMENTS[elem]['connectivity'][1])

In [11]:
### Writing Beam Elements File
abaqus_beam_str = '** This is Auto generated beam element details\n*Element, type=B31\n'
BEAMS = []
for key,value in EPR_mat.section_details['beam'].items():
    BEAMS = BEAMS + list(GROUPS[key]['group'])
for elem in BEAMS:
    if not(ELEMENTS[elem]['type'] == 21):
        print('Wrong element type {0:d}'.format(elem))
        continue
    abaqus_beam_str = abaqus_beam_str + '{0:d},{1:d},{2:d}\n'.format(elem,\
                                                ELEMENTS[elem]['connectivity'][0],\
                                                ELEMENTS[elem]['connectivity'][1])

In [12]:
"""for key,elemt in EPR_geo.fem['elements'][21].items():
    if key in BEAMS:
        continue
    elif key in SPRING_ELEMENTS_T+SPRING_ELEMENTS_TR:
        continue
    elif key in ROD_ELEMENTS:
        continue
    else:
        print('Orfan 21 Type Element - {0:d}'.format(key))"""

"for key,elemt in EPR_geo.fem['elements'][21].items():\n    if key in BEAMS:\n        continue\n    elif key in SPRING_ELEMENTS_T+SPRING_ELEMENTS_TR:\n        continue\n    elif key in ROD_ELEMENTS:\n        continue\n    else:\n        print('Orfan 21 Type Element - {0:d}'.format(key))"

In [13]:
abaqus_tri_shell_str = '** This is Auto generated Triangular Shell element details\n*Element, type=S3\n'
for key,value in EPR_geo.fem['elements'][91].items():
    abaqus_tri_shell_str = abaqus_tri_shell_str + '{0:d},{1:d},{2:d},{3:d}\n'.format(key,value[0],value[1],value[2])

In [14]:
abaqus_quad_shell_str = '** This is Auto generated Quadrilateral Shell element details\n*Element, type=S4R\n'
for key,value in EPR_geo.fem['elements'][94].items():
    abaqus_quad_shell_str = abaqus_quad_shell_str + '{0:d},{1:d},{2:d},{3:d},{4:d}\n'.format(key,value[0],value[1],value[2],value[3])

In [15]:
"""BAN
IN
BD2_W0V03
STRUC_SEG2
BAN_DT1D
BAN_DTLM
BAN_D2BD3M5
BAN_D5BD3M2"""

'BAN\nIN\nBD2_W0V03\nSTRUC_SEG2\nBAN_DT1D\nBAN_DTLM\nBAN_D2BD3M5\nBAN_D5BD3M2'

In [16]:
ABQUS_INP_HEADER ="""*HEADING
This Geometry Model Generated using Python Script
written by Anis M.V amv@aerb.gov.in\n"""

In [17]:
FINAL_INP_Str = ABQUS_INP_HEADER + abaqus_node_str + abaqus_truss_str + \
    abaqus_beam_str + abaqus_tri_shell_str + abaqus_quad_shell_str

In [18]:
with open('AERB_MODELS/Abaqus/geometry_without_group.inp','w') as f:
    f.write(FINAL_INP_Str)

In [19]:
with open('ELEMENTS.pickle', 'wb') as handle:
    pickle.dump(ELEMENTS, handle, protocol=2)
with open('NODES.pickle', 'wb') as handle:
    pickle.dump(EPR_geo.fem['nodes'], handle, protocol=2)

In [26]:
ELEMENTS[GROUPS['BRI_CNV_SIEC']['group'][0]]

{'type': 21, 'connectivity': [22521, 5147]}